In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from tqdm import tqdm
import seaborn as sns
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('final_dataset_backtest.csv')
df

,date,permno,ticker,prc,retx,shrout,cfacpr,vol,vwretd,vwretx,...,ou_forecast_20d,z_score_20d,fed_funds_rate,actual_vol_1d_lag1,actual_vol_5d,actual_vol_5d_lag1,actual_vol_10d,actual_vol_10d_lag1,actual_vol_20d,actual_vol_20d_lag1
0,1986-04-01,10008,GACO,-18.50,0.042254,2945.0,1.0,47.95,-0.011717,-0.011773,...,NaN,0.000000,7.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-04-02,10008,GACO,-18.00,-0.027027,2945.0,1.0,231.00,0.001289,0.001186,...,-0.002341,-0.753421,7.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1986-04-03,10008,GACO,-18.25,0.013889,2945.0,1.0,3.50,-0.009560,-0.009571,...,-0.002341,0.365600,7.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1986-04-04,10008,GACO,-18.25,0.000000,2945.0,1.0,39.50,-0.013887,-0.014239,...,-0.002341,-0.261429,6.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-04-07,10008,GACO,-18.25,0.000000,2945.0,1.0,68.00,-0.002678,-0.002752,...,-0.002341,-0.143432,7.09,NaN,0.056338,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14686405,2024-12-24,93436,TSLA,462.28,0.073572,3210060.0,1.0,593515.06,0.010566,0.010521,...,0.001722,-0.114998,4.33,NaN,0.131718,0.106439,0.157805,0.145334,0.170305,0.167895
14686406,2024-12-26,93436,TSLA,454.13,-0.017630,3210060.0,1.0,763922.73,0.000346,0.000282,...,0.001722,-0.317544,4.33,NaN,0.095352,0.131718,0.152724,0.157805,0.172878,0.170305
14686407,2024-12-27,93436,TSLA,431.66,-0.049479,3210060.0,1.0,823703.45,-0.010692,-0.010775,...,0.001722,0.117134,4.33,NaN,0.111126,0.095352,0.161946,0.152724,0.182332,0.172878
14686408,2024-12-30,93436,TSLA,417.41,-0.033012,3210060.0,1.0,647054.52,-0.009878,-0.009900,...,0.001722,0.186010,4.33,NaN,0.110521,0.111126,0.159511,0.161946,0.186401,0.182332


In [3]:
# Column-name helper (to get relevant columns based on the horizon (1/5/10/20 days)
def get_forecast_columns(horizon:int, include_actual_vol:bool=False):
    garch  = f"garch_vol_{horizon}d_lag1"
    ou     = f"ou_forecast_{horizon}d"
    z      = f"z_score_{horizon}d"
    if include_actual_vol:
        av        = f"actual_vol_{horizon}d"
        av_lag    = f"{av}_lag1"
        return garch, ou, z, av, av_lag
    return garch, ou, z

In [4]:
def prepare_trade_data(df: pd.DataFrame,
                       horizon: int = 5,
                       z_threshold: float = 1.5,
                       keep_actual_vol: bool = False) -> pd.DataFrame:
    """
    Clean the raw DataFrame so it is ready for the back-test.

    Parameters
    ----------
    df              : raw input DataFrame
    horizon         : forecast horizon in days (1 / 5 / 10 / 20 …)
    keep_actual_vol : if True also require the realised-volatility columns
                      (actual_vol_{horizon}d and its lag-1)

    Returns
    -------
    df_clean : tidy DataFrame with
               • date → datetime64
               • no NaNs in the columns needed by the trading engine
    """
    
    # --- figure-out which columns are needed ------------------------------
    garch_col, ou_col, z_col = get_forecast_columns(horizon)
    required_cols = ['date', 'permno', 'group_id', 'adj_prc',
                     garch_col, ou_col, z_col]

    if keep_actual_vol:
        _, _, _, act_vol, act_vol_lag = get_forecast_columns(horizon,
                                                             include_actual_vol=True)
        required_cols += [act_vol, act_vol_lag]

    # --- validate the DataFrame -------------------------------------------
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required column(s) for horizon {horizon}: {missing}")
    
    # --- drop rows with any NaN in those columns ---------------------------
    df_clean = (df
                .dropna(subset=required_cols)
                .copy())

    # --- make sure 'date' is proper datetime -------------------------------
    if not pd.api.types.is_datetime64_any_dtype(df_clean['date']):
        df_clean['date'] = pd.to_datetime(df_clean['date'])

    # Vectorized signal assignment
    df_clean['signal'] = 0
    df_clean.loc[df_clean[z_col] <= -z_threshold, 'signal'] = 1    # Long signal
    df_clean.loc[df_clean[z_col] >= z_threshold, 'signal'] = -1    # Short signal

    return df_clean

In [5]:
df.isnull().sum()

date                                  0
permno                                0
ticker                             1346
prc                                   0
retx                                  0
shrout                                0
cfacpr                                0
vol                                   0
vwretd                                0
vwretx                                0
trading_start                         0
group_id                              0
adv20                                 0
adj_prc                               0
group_avg_retx                        0
retx_relative                         0
retx_relative_lag1                12441
group_std_retx_relative_1d     14686410
z_score_classic_1d                    0
z_score_classic_1d_lag1               0
group_std_retx_relative_5d            0
z_score_classic_5d                    0
z_score_classic_5d_lag1               0
group_std_retx_relative_10d           0
z_score_classic_10d                   0


In [6]:
# Constants (parameters)
TCOST_PER_SHARE = 0.10          # $0.10 in & $0.10 out
FIN_SPREAD_LONG = 0.015         # +1.5 % over fed-funds when long
FIN_SPREAD_SHORT = 0.010        # +1.0 % rebate when short
CALENDAR_DAYS   = 365           # for financing accrual
TRADING_DAYS    = 252           # for Sharpe etc.

In [7]:
# Handle cash, costs, portfolios and stock weights
class Finance:
    def __init__(self, initial_aum: float, garch_col: str, ou_col: str):
        self.cash = initial_aum
        self.positions = {}
        self.port_w = {}
        self.prev_q = None
        self.garch_col = garch_col
        self.ou_col = ou_col

    def debit(self, amount: float):
        self.cash = max(self.cash - amount, 0)

    def credit(self, amount: float):
        self.cash += amount

    def calc_cost(self, side: int, shares: int, price: float, ff_rate: float, event: str):
        spread = FIN_SPREAD_LONG if side == 1 else FIN_SPREAD_SHORT
        fin_cost = t_cost = 0.0
        if event in ("open", "hold"):
            fin_cost = price * shares * ((ff_rate + spread) / CALENDAR_DAYS)
        if event in ("open", "close"):
            t_cost = TCOST_PER_SHARE
        return fin_cost, t_cost

    # ────────────────────────────────────────────────────────────────────────────────
    def quarter_weights(self, day_slice: pd.DataFrame, method: str = 'inv_vol', short_bias_factor: float = 1.5):
        """
        Portfolio-level capital allocation with optional short bias.
        
        - method: 'inv_vol' (Inverse Volatility), 'rsk_par' (Risk Parity), 'mean_var' (Mean-Variance)
        - short_bias_factor: Multiplier to favor groups with more short signals (default: 1.5)
        """
        risk = day_slice.groupby('group_id')[self.garch_col].mean()
        groups = risk.index.tolist()
    
        # Calculate the average signal per group to detect short dominance
        signal_avg = day_slice.groupby('group_id')['signal'].mean()  # Negative avg → more shorts
    
        # Compute bias adjustment: Increase weight for groups with negative average signals
        bias_adjustment = 1 + (-signal_avg.clip(upper=0)) * (short_bias_factor - 1)
        # Example: 
        # If avg signal = -1 → bias_adjustment = 1 + (1) * (1.5 - 1) = 1.5
        # If avg signal = 0.5 → bias_adjustment = 1 + 0 = 1
    
        if method in ['inv_vol', 'rsk_par']:
            inv = 1 / risk.replace(0, np.nan)
            adjusted_weights = inv * bias_adjustment
            return (adjusted_weights / adjusted_weights.sum()).to_dict()
    
        elif method == 'mean_var':
            mu = day_slice.groupby('group_id')[self.ou_col].mean().values
            cov = np.diag(risk.values ** 2)
    
            n = len(mu)
            def objective(w, lam=0.1):
                return w @ cov @ w - lam * (w @ mu)
    
            constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
            bounds = [(0, 1) for _ in range(n)]
    
            res = minimize(objective, x0=np.ones(n) / n, bounds=bounds, constraints=constraints)
            weights = res.x if res.success else np.ones(n) / n
            weights *= bias_adjustment.values  # Apply bias adjustment to optimized weights
            weights /= weights.sum()  # Normalize to sum to 1
    
            return dict(zip(groups, weights))
    
        else:
            raise ValueError(f"Unknown weighting method: {method}")


    # ────────────────────────────────────────────────────────────────────────────────
    def stock_weights(self, group_slice: pd.DataFrame, method: str = 'inv_vol'):
        """
        Stock-level capital allocation within a group:
        - method: 'inv_vol', 'rsk_par', 'mean_var'
        """
        score = group_slice[self.ou_col] / group_slice[self.garch_col]
        score = score.replace([np.inf, -np.inf], np.nan).fillna(0)

        long_m = (group_slice['signal'] == 1) & (score > 0)
        short_m = (group_slice['signal'] == -1) & (score < 0)

        w = {}
        if method == 'inv_vol' or method == 'rsk_par':
            if long_m.any():
                s = score[long_m]
                w.update(dict(zip(
                    group_slice.loc[long_m, 'permno'], s / s.sum()
                )))
            if short_m.any():
                s = -score[short_m]
                w.update(dict(zip(
                    group_slice.loc[short_m, 'permno'], s / s.sum()
                )))
            return w

        elif method == 'mean_var':
            mu = group_slice[self.ou_col].values
            sigma = group_slice[self.garch_col].values
            cov = np.diag(sigma ** 2)

            n = len(mu)
            def objective(weights, lam=0.1):
                return weights @ cov @ weights - lam * (weights @ mu)

            constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]
            bounds = [(0, 1) for _ in range(n)]

            res = minimize(objective, x0=np.ones(n) / n, bounds=bounds, constraints=constraints)
            if res.success:
                return dict(zip(group_slice['permno'], res.x))
            else:
                return dict(zip(group_slice['permno'], np.ones(n) / n))

        else:
            raise ValueError(f"Unknown weighting method: {method}")

In [8]:
# ────────────────────────────────────────────────────────────────────────────────
def should_invest_and_shares(alloc, price, adv, liquidity_cap=0.1):
    """
    Decide if investment should be made and calculate shares to trade.

    Parameters:
    - alloc: Allocated capital for the stock.
    - price: Current price of the stock.
    - adv: 20-day average daily volume.
    - liquidity_cap: Max percentage of ADV allowed to trade (default: 10%).

    Returns:
    - should_invest (bool): True if the position meets liquidity requirements.
    - shares (int): Number of shares to trade.
    """
    shares_cap = int(adv * liquidity_cap)
    shares = int(alloc / price)
    shares = min(shares, shares_cap)

    return (shares > 0), shares

def exit_signal(z: float, side: int, thr: float, pnl_pct: float,
                hold_days: int, max_hold: int, min_profit: float) -> bool:
    """
    Exit decision rule combining z-score, profit threshold, and max holding period.

    Parameters
    ----------
    z          : Current z–score.
    side       : +1 for long, –1 for short.
    thr        : Z-score threshold for exit signal.
    pnl_pct    : Current % PnL (signed based on position side).
    hold_days  : Number of days the position has been held.
    max_hold   : Maximum holding days allowed before forced exit.
    min_profit : Minimum % profit required to exit on z-signal (default 10%).

    Returns
    -------
    bool : True if the position should be exited.
    """

    # 1. Exit if z-score suggests mean reversion AND profit condition is met
    z_exit = (side == 1 and z < thr) or (side == -1 and z > -thr)
    profit_exit = pnl_pct >= min_profit
    exit_on_signal = z_exit and profit_exit

    if exit_on_signal:
        return True, 'z_signal_profit'
        
    if hold_days >= max_hold:
        return True, 'max_hold'
    
    return False, 'none'  # Keep position open

In [9]:
# Back-test (only 3 args!)
def backtest(df: pd.DataFrame, 
              aum: float = 100_000_000.0, 
              horizon: int = 5, 
              z_threshold: float = 1.5, 
              portfolio_weight_method: str = 'inv_vol', 
              stock_weight_method: str = 'inv_vol',
             max_hold: int = 5, min_profit: float = 0.05):
    
    df = prepare_trade_data(df, horizon=horizon, z_threshold = z_threshold)
    
    # df columns cleaned. Example: ['date', 'permno', 'group_id', 'adj_prc', 'z_score_5d', ...]
    print('Data prep complete')
    garch_col, ou_col, z_col = get_forecast_columns(horizon)
    print("column names: ", garch_col, ou_col, z_col)
    # Example: garch_col = 'garch_vol_5d_lag1', ou_col = 'ou_forecast_5d', z_col = 'z_score_5d'

    ff_col = 'fed_funds_rate'

    df = df.sort_values('date').copy()
    # df is now sorted by date. Example: df['date'].head() → [1975-01-02, 1975-01-02, 1975-01-02, ...]

    df['date'] = pd.to_datetime(df['date'])
    # Ensures 'date' is datetime. Example: df['date'].dtype → datetime64[ns]

    fin = Finance(aum, garch_col, ou_col)
    # Example: fin.cash → 100,000,000.0

    log = []
    
    for date, day in tqdm(df.groupby('date', sort=True), total=df['date'].nunique()):
        # Example: date = Timestamp('1975-01-02')
        #          day = DataFrame with today's trading data
        
        if date.to_period('Q') != fin.prev_q:
            fin.prev_q = date.to_period('Q')
            fin.port_w = fin.quarter_weights(day, method = portfolio_weight_method, 
                                             short_bias_factor = 1.5)
            # Example: fin.port_w = {'Tech': 0.25, 'Finance': 0.35, 'Energy': 0.40}

        price = day.set_index('permno')['adj_prc'].to_dict()
        # Example: price = {10001: 45.5, 10002: 60.3, 10003: 75.2}

        z = day.set_index('permno')[z_col].to_dict()
        # Example: z = {10001: 1.6, 10002: -2.0, 10003: 0.8}

        ff = day['fed_funds_rate'].mean()
        # Example: ff = 5.5

        # Exit Logic
        for perm_id, position in list(fin.positions.items()):
        
            if perm_id not in price:
                continue
        
            px   = price[perm_id]
            zz   = z.get(perm_id, np.nan)
            ffD  = ff
            side = position['side']
        
            # Daily financing ---------------------------------------------------
            fin_cost, _ = fin.calc_cost(side, position['shares_unit'], px, ffD, 'hold')
            fin.debit(fin_cost)
            position['financing_cost_total'] += fin_cost
            position['total_carrying_cost']      += fin_cost
        
            pnl_pct   = (px - position['entry_price']) / (position['entry_price'] + position['total_carrying_cost']) * side
            hold_days = (date - position['entry_date']).days
        
            should_exit, exit_reason = exit_signal(zz, side, z_threshold, pnl_pct, hold_days, 
                                                   max_hold, min_profit)
            if not should_exit:
                continue  # Keep position open 
                
            # Transaction cost to close ----------------------------------------
            _, t_cost = fin.calc_cost(side, position['shares_unit'], px, ffD, 'close')
            fin.debit(t_cost)
            position['transaction_cost_close'] = t_cost
            position['total_carrying_cost']       += t_cost
        
            # Cash-flow on close -----------------------------------------------
            if side == 1:                                    # LONG
                proceeds  = position['shares_unit'] * px
                fin.credit(proceeds)
                gross_pnl = proceeds - position['principal_investment']
            else:                                            # SHORT
                buyback   = position['shares_unit'] * px
                fin.debit(buyback)
                proceeds  = position['principal_investment']               # cash received at entry
                gross_pnl = proceeds - buyback
        
            # Re-compute AUM **after** the trade --------------------------------
            current_aum = fin.cash + sum(
                open_pos['side'] * open_pos['shares_unit'] *
                price.get(pn, px)            # px fallback if intraday missing
                for pn, open_pos in fin.positions.items()
            )
        
            # Log trade ---------------------------------------------------------
            log.append({
                **position,
                'exit_date'     : date,
                'exit_price'    : px,
                'gross_pnl'     : gross_pnl,
                'net_pnl'       : gross_pnl - position['total_carrying_cost'],
                'holding_days'  : hold_days,
                'gross_return'  : pnl_pct,
                'net_return'    : (gross_pnl - position['total_carrying_cost']) /
                                  abs(position['principal_investment']),
                'exit_reason'   : exit_reason,
                'status'        : 'closed',
                'aum_after'     : current_aum
            })
        
            del fin.positions[perm_id]

        
        # Entry logic
        for gid, grp in day.groupby('group_id'):
        
            sig = grp[grp['signal'] != 0]
            if sig.empty:
                continue
        
            cap_grp = fin.cash * fin.port_w.get(gid, 0)        # $ allocated to group
            weights = fin.stock_weights(sig, method = stock_weight_method)                   # {permno: w}
        
            for _, row in sig.iterrows():
                perm = row.permno
                if perm in fin.positions or weights.get(perm, 0) == 0:
                    continue
            
                alloc = cap_grp * weights[perm]  # Allocated capital to this stock
                should_invest, sh = should_invest_and_shares(alloc, row.adj_prc, row.adv20)
            
                if not should_invest:
                    continue  # Skip this stock if investment isn't feasible under liquidity constraint
            
                side = int(np.sign(row.signal))
                px = row.adj_prc
        
                # *** entry day: transaction cost only, NO financing yet ***
                fin_cost_open = 0.0
                _, t_cost_open = fin.calc_cost(side, sh, px, ff, 'open')
        
                if side == 1:          # LONG → we pay for the shares up-front
                    cash_need = sh * px + t_cost_open          # shares + open cost
                    if fin.cash < cash_need:
                        continue
                    fin.debit(cash_need)                       # ↓ cash
                    invested_or_proceeds = sh * px             # store purchase cost
                else:                  # SHORT → we *receive* sale proceeds today
                    proceeds = sh * px
                    fin.credit(proceeds)                       # ↑ cash
                    fin.debit(t_cost_open)                     # pay open cost
                    invested_or_proceeds = proceeds            # store sale proceeds
        
                # record position
                fin.positions[perm] = dict(
                    entry_date            = date,
                    group_id              = gid,
                    side                  = side,
                    entry_price           = px,
                    shares_unit           = sh,
                    principal_investment  = invested_or_proceeds,
                    financing_cost_total  = 0.0,
                    transaction_cost_open = t_cost_open,
                    transaction_cost_close= 0.0,  # fill on exit
                    total_carrying_cost   = t_cost_open,  # Starts with open transaction cost
                    status                = 'open'
                )


    # Append still-open positions to log
    for perm, pos in fin.positions.items():
        log.append({**pos, 'exit_date': pd.NaT, 'exit_price': np.nan, 
                    'transaction_cost_close': 0.0, 'status': 'open'})

    trades = pd.DataFrame(log)

    print(f"Final cash: ${fin.cash:,.0f}")
    return trades

In [10]:
def summarize_positions(trades: pd.DataFrame):
    """
    Summarizes the trade book by side (Long/Short) and status (Open/Closed).
    Prints count, total investment, and total PnL.

    Parameters:
    - trades: DataFrame containing trade records with columns:
      ['entry_date', 'group_id', 'side', 'entry_price', 'sh', 'invest', 
       'fin_cost', 'transaction_cost_open', 'status', 'exit_date', 
       'exit_price', 'transaction_cost_close', 'gross_pnl']
    """
    summary = []

    for side_val, side_name in [(-1, 'Short'), (1, 'Long')]:
        for status_val in ['open', 'closed']:
            filtered = trades[(trades['side'] == side_val) & (trades['status'] == status_val)]
            
            count = len(filtered)
            total_investment = filtered['principal_investment'].sum() + filtered['total_carrying_cost'].sum()
            total_pnl = filtered['net_pnl'].sum() if 'net_pnl' in filtered.columns else 0.0

            summary.append({
                'Side': side_name,
                'Status': status_val.capitalize(),
                'Count': count,
                'Total Investment': f"${total_investment:,.2f}",
                'Total PnL': f"${total_pnl:,.2f}"
            })

    summary_df = pd.DataFrame(summary)
    display(summary_df)  
    return summary_df

In [11]:
trades_10 = backtest(df, aum=100_000_000_000, horizon=1, z_threshold=1.2, 
                     portfolio_weight_method='rsk_par', stock_weight_method='rsk_par', 
                     max_hold = 25, min_profit = 0.08)

trades_10

Data prep complete
column names:  garch_vol_1d_lag1 ou_forecast_1d z_score_1d


100%|█████████████████████████████████████| 12546/12546 [03:05<00:00, 67.64it/s]


Final cash: $55,780,373,887


,entry_date,group_id,side,entry_price,shares_unit,principal_investment,financing_cost_total,transaction_cost_open,transaction_cost_close,total_carrying_cost,status,exit_date,exit_price,gross_pnl,net_pnl,holding_days,gross_return,net_return,exit_reason,aum_after
0,1975-04-08,1975-Q1-01,1,6.562500,7,45.937500,2.196148,0.1,0.1,2.396148,closed,1975-04-11,7.437500,6.125000,3.728852,3.0,0.098774,0.081172,z_signal_profit,1.000000e+11
1,1975-04-10,1975-Q1-08,1,1.066703,10,10.667029,0.881765,0.1,0.1,1.081765,closed,1975-04-17,1.231870,1.651669,0.569904,7.0,0.080629,0.053427,z_signal_profit,1.000000e+11
2,1975-04-17,1975-Q1-08,-1,2.890625,7,20.234375,0.867619,0.1,0.1,1.067619,closed,1975-04-22,2.578125,2.187500,1.119881,5.0,0.080995,0.055345,z_signal_profit,1.000000e+11
3,1975-04-02,1975-Q1-01,-1,0.799410,40,31.976395,9.412745,0.1,0.1,9.612745,closed,1975-04-28,0.890885,-3.658990,-13.271736,26.0,-0.008871,-0.415048,max_hold,1.000000e+11
4,1975-04-02,1975-Q1-07,1,12.062500,55,663.437500,217.464208,0.1,0.1,217.664208,closed,1975-04-28,15.500000,189.062500,-28.601708,26.0,0.014970,-0.043111,max_hold,1.000000e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365558,2024-12-31,2024-Q3-00,1,4.980000,124,617.520000,0.000000,0.1,0.0,0.100000,open,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365559,2024-12-31,2024-Q3-03,1,1.160000,383,444.280000,0.000000,0.1,0.0,0.100000,open,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365560,2024-12-31,2024-Q3-04,-1,0.750000,160,120.000000,0.000000,0.1,0.0,0.100000,open,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365561,2024-12-31,2024-Q3-04,1,2.380000,121,287.980000,0.000000,0.1,0.0,0.100000,open,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
summary_table_exit = trades_10.groupby(['exit_reason', 'side']).agg(
    total_count=('side', 'count'),
    total_net_pnl=('net_pnl', 'sum'),
    total_principal_investment=('principal_investment', 'sum'),
    total_carrying_cost=('total_carrying_cost', 'sum')
).reset_index()

summary_table_exit['total_investment_sum'] = summary_table_exit['total_principal_investment'] + summary_table_exit['total_carrying_cost']

# Optional: Replace side values for better readability if side is stored as 1/-1
summary_table_exit['side'] = summary_table_exit['side'].map({1: 'Long', -1: 'Short'})

print('Summary based on exit reason')
summary_table_exit

Summary based on exit reason


,exit_reason,side,total_count,total_net_pnl,total_principal_investment,total_carrying_cost,total_investment_sum
0,max_hold,Short,129474,-9.045648e+09,5.778231e+10,7.098219e+09,6.488053e+10
1,max_hold,Long,187608,-3.697100e+10,1.525412e+11,1.339338e+10,1.659345e+11
2,z_signal_profit,Short,24923,6.207541e+08,2.761995e+09,2.364094e+07,2.785636e+09
3,z_signal_profit,Long,18398,1.934090e+09,6.962642e+09,7.342175e+07,7.036064e+09


In [13]:
print('summary based on open/close position')
summary = summarize_positions(trades_10)

summary based on open/close position


,Side,Status,Count,Total Investment,Total PnL
0,Short,Open,2426,"$303,695,666.72",$0.00
1,Short,Closed,154397,"$67,666,166,516.72","$-8,424,893,411.55"
2,Long,Open,2734,"$1,037,573,672.63",$0.00
3,Long,Closed,206006,"$172,970,611,341.86","$-35,036,910,241.50"


In [14]:
# Add Year Column
trades_10['year'] = trades_10['entry_date'].dt.year

# summary calculation function
def calculate_summary(df_year):
    # Filter out NaT and ensure dates belong to the current year
    entry_dates = df_year['entry_date'].dropna().dt.normalize()
    exit_dates = df_year['exit_date'].dropna().dt.normalize()

    # Keep only dates within the year being processed
    year_value = df_year['year'].iloc[0]
    entry_dates = entry_dates[entry_dates.dt.year == year_value]
    exit_dates = exit_dates[exit_dates.dt.year == year_value]

    # Calculate unique trading days correctly
    trading_days = pd.concat([entry_dates, exit_dates]).drop_duplicates().nunique()

    
    gross_return_mean = df_year['net_pnl'].mean()
    gross_return_std = df_year['net_pnl'].std(ddof=0)

    sharpe_ratio = (gross_return_mean / gross_return_std) * np.sqrt(trading_days) if gross_return_std != 0 else np.nan

    return pd.Series({
        'num_long': (df_year['side'] == 1).sum(),
        'num_short': (df_year['side'] == -1).sum(),
        'total_trades': len(df_year),
        'total_principal_investment': df_year['principal_investment'].sum(),
        'financing_cost': df_year['financing_cost_total'].sum(),
        'transaction_entry_cost': df_year['transaction_cost_open'].sum(),
        'transaction_exit_cost': df_year['transaction_cost_close'].sum(),
        'total_carrying_cost': df_year[['transaction_cost_open', 'transaction_cost_close', 'financing_cost_total']].sum(axis=1).sum(),
        'total_income': df_year['exit_price'].multiply(df_year['shares_unit'], fill_value=0).sum(),
        'trading_days': trading_days,
        'sharpe_ratio': sharpe_ratio,
        #'total_closed': (df_year['status'] == 'closed').sum(),
        'not_closed': (df_year['status'] == 'open').sum(),
        'net_pnl': (df_year['net_pnl'].sum())
    })

# Group by Year and Apply Summary Function
summary_table = trades_10.groupby('year').apply(calculate_summary).reset_index()

# Round Sharpe Ratio for readability
summary_table['sharpe_ratio'] = summary_table['sharpe_ratio'].round(2)

# Final Summary Table
summary_table

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_24169/776679137.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_table = trades_10.groupby('year').apply(calculate_summary).reset_index()


,year,num_long,num_short,total_trades,total_principal_investment,financing_cost,transaction_entry_cost,transaction_exit_cost,total_carrying_cost,total_income,trading_days,sharpe_ratio,not_closed,net_pnl
0,1975,191.0,162.0,353.0,8.068542e+04,2.100496e+04,35.3,35.2,2.107546e+04,7.894497e+04,175.0,-3.93,1.0,-1.958880e+04
1,1976,267.0,217.0,484.0,1.016465e+05,2.293236e+04,48.4,48.0,2.302876e+04,9.973651e+04,232.0,-6.17,4.0,-2.258737e+04
2,1977,137.0,134.0,271.0,3.950502e+05,8.936822e+04,27.1,26.9,8.942222e+04,3.733089e+05,196.0,-1.52,2.0,-1.084416e+05
3,1978,207.0,146.0,353.0,1.018062e+05,4.238208e+04,35.3,35.2,4.245258e+04,1.079953e+05,192.0,-3.03,1.0,-4.780081e+04
4,1979,250.0,137.0,387.0,2.947115e+05,1.618348e+05,38.7,38.4,1.619119e+05,2.989167e+05,199.0,-4.96,3.0,-1.663871e+05
5,1980,530.0,270.0,800.0,6.984516e+05,4.707634e+05,80.0,79.9,4.709233e+05,7.096941e+05,241.0,-2.72,1.0,-4.731593e+05
6,1981,768.0,472.0,1240.0,1.163794e+07,9.455494e+06,124.0,122.3,9.455740e+06,1.190816e+07,252.0,-1.26,17.0,-8.845658e+06
7,1982,452.0,306.0,758.0,8.824551e+06,5.541135e+06,75.8,75.2,5.541286e+06,9.548992e+06,241.0,-1.49,6.0,-6.214786e+06
8,1983,1878.0,1342.0,3220.0,1.141848e+07,5.036732e+06,322.0,318.5,5.037373e+06,1.133761e+07,253.0,-1.00,35.0,-5.308123e+06
9,1984,1831.0,1632.0,3463.0,1.084416e+07,5.177211e+06,346.3,341.4,5.177898e+06,9.601118e+06,253.0,-1.01,49.0,-4.532745e+06


In [15]:
# Add Year and Quarter Columns
trades_10['year'] = trades_10['entry_date'].dt.year
trades_10['quarter'] = trades_10['entry_date'].dt.quarter

# Group Identifier for Year-Quarter
trades_10['year_quarter'] = trades_10['year'].astype(str) + ' Q' + trades_10['quarter'].astype(str)

# Group by Year-Quarter and Apply Summary Function
quarterly_summary = trades_10.groupby('year_quarter').apply(calculate_summary).reset_index()

# Round Sharpe Ratio for readability
quarterly_summary['sharpe_ratio'] = quarterly_summary['sharpe_ratio'].round(2)

# Final Quarterly Summary
quarterly_summary

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_24169/3545530602.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quarterly_summary = trades_10.groupby('year_quarter').apply(calculate_summary).reset_index()


,year_quarter,num_long,num_short,total_trades,total_principal_investment,financing_cost,transaction_entry_cost,transaction_exit_cost,total_carrying_cost,total_income,trading_days,sharpe_ratio,not_closed,net_pnl
0,1975 Q2,68.0,54.0,122.0,2.709024e+04,7.036825e+03,12.2,12.2,7.061225e+03,2.749257e+04,66.0,-2.99,0.0,-6.665529e+03
1,1975 Q3,75.0,75.0,150.0,2.685748e+04,7.421469e+03,15.0,14.9,7.451369e+03,2.509403e+04,70.0,-3.62,1.0,-7.409459e+03
2,1975 Q4,48.0,33.0,81.0,2.673770e+04,6.546668e+03,8.1,8.1,6.562868e+03,2.635837e+04,56.0,-1.65,0.0,-5.513816e+03
3,1976 Q1,60.0,56.0,116.0,2.245257e+04,4.944786e+03,11.6,11.6,4.967986e+03,2.262353e+04,70.0,-5.26,0.0,-5.420593e+03
4,1976 Q2,77.0,52.0,129.0,2.033228e+04,4.915145e+03,12.9,12.8,4.940845e+03,1.991181e+04,65.0,-4.41,1.0,-5.162359e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2023 Q4,737.0,793.0,1530.0,2.875993e+08,7.315461e+07,153.0,151.5,7.315492e+07,3.010533e+08,63.0,-1.38,15.0,-7.514234e+07
195,2024 Q1,842.0,643.0,1485.0,3.411742e+08,8.617211e+07,148.5,145.3,8.617240e+07,3.453007e+08,73.0,-1.82,32.0,-8.265295e+07
196,2024 Q2,847.0,772.0,1619.0,3.474049e+08,8.782952e+07,161.9,156.5,8.782984e+07,3.502419e+08,75.0,-1.70,54.0,-8.508651e+07
197,2024 Q3,812.0,690.0,1502.0,4.071802e+08,1.018730e+08,150.2,145.9,1.018733e+08,4.080070e+08,75.0,-1.55,43.0,-9.318552e+07


In [16]:
# Add Date Column for Grouping
trades_10['trade_day'] = trades_10['entry_date'].dt.date

# Group by Each Trading Day and Apply Summary Function
daily_summary = trades_10.groupby('trade_day').apply(calculate_summary).reset_index()

# Round Sharpe Ratio for readability
daily_summary['sharpe_ratio'] = daily_summary['sharpe_ratio'].round(2)

# Final Daily Summary
daily_summary

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_24169/4218421627.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = trades_10.groupby('trade_day').apply(calculate_summary).reset_index()


,trade_day,num_long,num_short,total_trades,total_principal_investment,financing_cost,transaction_entry_cost,transaction_exit_cost,total_carrying_cost,total_income,trading_days,sharpe_ratio,not_closed,net_pnl
0,1975-04-02,1.0,1.0,2.0,6.954139e+02,226.876953,0.2,0.2,227.276953,888.135386,2.0,-3.86,0.0,-41.873444
1,1975-04-03,1.0,0.0,1.0,1.135333e+03,308.576429,0.1,0.1,308.776429,1282.666667,2.0,NaN,0.0,-161.443096
2,1975-04-04,3.0,0.0,3.0,6.791135e+02,177.682032,0.3,0.3,178.282032,703.505287,2.0,-2.75,0.0,-153.890268
3,1975-04-07,1.0,1.0,2.0,5.286039e+02,155.779367,0.2,0.2,156.179367,547.056070,2.0,-1.51,0.0,-165.643873
4,1975-04-08,2.0,1.0,3.0,8.058456e+01,12.817295,0.3,0.3,13.417295,89.022057,3.0,-0.71,0.0,-4.979795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,2024-12-24,9.0,1.0,10.0,2.887815e+06,135199.003901,1.0,0.0,135200.003901,0.000000,1.0,NaN,10.0,0.000000
12147,2024-12-26,5.0,2.0,7.0,1.878440e+05,6485.622230,0.7,0.1,6486.422230,388.800000,2.0,NaN,6.0,99.121682
12148,2024-12-27,13.0,4.0,17.0,5.143813e+04,1244.640632,1.7,0.0,1246.340632,0.000000,1.0,NaN,17.0,0.000000
12149,2024-12-30,7.0,2.0,9.0,3.133200e+04,368.675343,0.9,0.0,369.575343,0.000000,1.0,NaN,9.0,0.000000


In [17]:
print('total: ', trades_10['net_pnl'].sum())
print('annual: ', summary_table['net_pnl'].sum())
print('quaterly: ', quarterly_summary['net_pnl'].sum())
print('daily: ', daily_summary['net_pnl'].sum())

total:  -43461803653.050224
annual:  -43461803653.050224
quaterly:  -43461803653.05022
daily:  -43461803653.05022


In [18]:
def select_columns(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Returns a DataFrame with only the specified columns.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - columns (list): List of column names to select.
    
    Returns:
    - pd.DataFrame: Filtered DataFrame with specified columns.
    """
    missing_cols = [col for col in columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"The following columns are missing in the DataFrame: {missing_cols}")
    
    return df[columns]

In [20]:
trades_10['total_investment'] = trades_10['principal_investment'] + trades_10['total_carrying_cost']
columns = ['entry_date', 'group_id', 'side', 'entry_price', 'shares_unit', 'exit_date', 'exit_price',
           'principal_investment', 'total_carrying_cost', 'total_investment',
           'gross_pnl', 'net_pnl', 'exit_reason']

trade_log = select_columns(trades_10, columns)
print('Trade log of growth portfolio with trend following method')
trade_log

Trade log of growth portfolio with trend following method


,entry_date,group_id,side,entry_price,shares_unit,exit_date,exit_price,principal_investment,total_carrying_cost,total_investment,gross_pnl,net_pnl,exit_reason
0,1975-04-08,1975-Q1-01,1,6.562500,7,1975-04-11,7.437500,45.937500,2.396148,48.333648,6.125000,3.728852,z_signal_profit
1,1975-04-10,1975-Q1-08,1,1.066703,10,1975-04-17,1.231870,10.667029,1.081765,11.748794,1.651669,0.569904,z_signal_profit
2,1975-04-17,1975-Q1-08,-1,2.890625,7,1975-04-22,2.578125,20.234375,1.067619,21.301994,2.187500,1.119881,z_signal_profit
3,1975-04-02,1975-Q1-01,-1,0.799410,40,1975-04-28,0.890885,31.976395,9.612745,41.589141,-3.658990,-13.271736,max_hold
4,1975-04-02,1975-Q1-07,1,12.062500,55,1975-04-28,15.500000,663.437500,217.664208,881.101708,189.062500,-28.601708,max_hold
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365558,2024-12-31,2024-Q3-00,1,4.980000,124,NaT,NaN,617.520000,0.100000,617.620000,NaN,NaN,NaN
365559,2024-12-31,2024-Q3-03,1,1.160000,383,NaT,NaN,444.280000,0.100000,444.380000,NaN,NaN,NaN
365560,2024-12-31,2024-Q3-04,-1,0.750000,160,NaT,NaN,120.000000,0.100000,120.100000,NaN,NaN,NaN
365561,2024-12-31,2024-Q3-04,1,2.380000,121,NaT,NaN,287.980000,0.100000,288.080000,NaN,NaN,NaN


In [ ]:
trades_10.to_csv('test1.csv', index = False)

In [ ]:
summary_table.columns

In [ ]:
summary_table['total_investment'] = summary_table['total_principal_investment'] + summary_table['total_carrying_cost']
columns = ['year', 'num_long', 'num_short', 'total_principal_investment', 'total_carrying_cost', 'total_investment', 'trading_days', 
           'sharpe_ratio', 'not_closed', 'net_pnl']

annual_table = select_columns(summary_table, columns)
print('Yearly report of growth portfolio (trend following)')
annual_table

In [45]:
quarterly_summary.columns

Index(['year_quarter', 'num_long', 'num_short', 'total_trades',
       'total_principal_investment', 'financing_cost',
       'transaction_entry_cost', 'transaction_exit_cost',
       'total_carrying_cost', 'total_income', 'trading_days', 'sharpe_ratio',
       'not_closed', 'net_pnl', 'total_investment'],
      dtype='object')

In [46]:
quarterly_summary['total_investment'] = quarterly_summary['total_principal_investment'] + quarterly_summary['total_carrying_cost']
columns = ['year_quarter', 'num_long', 'num_short', 'total_principal_investment', 'total_carrying_cost', 'total_investment', 'trading_days', 
           'sharpe_ratio', 'not_closed', 'net_pnl']

quaterly_table = select_columns(quarterly_summary, columns)
print('Quaterly report of growth portfolio (trend following')
quaterly_table

Quaterly report of growth portfolio (trend following


,year_quarter,num_long,num_short,total_principal_investment,total_carrying_cost,total_investment,trading_days,sharpe_ratio,not_closed,net_pnl
0,1975 Q2,68.0,54.0,2.709024e+04,7.061225e+03,3.415147e+04,66.0,-2.99,0.0,-6.665529e+03
1,1975 Q3,75.0,75.0,2.685748e+04,7.451369e+03,3.430885e+04,70.0,-3.62,1.0,-7.409459e+03
2,1975 Q4,48.0,33.0,2.673770e+04,6.562868e+03,3.330057e+04,56.0,-1.65,0.0,-5.513816e+03
3,1976 Q1,60.0,56.0,2.245257e+04,4.967986e+03,2.742056e+04,70.0,-5.26,0.0,-5.420593e+03
4,1976 Q2,77.0,52.0,2.033228e+04,4.940845e+03,2.527313e+04,65.0,-4.41,1.0,-5.162359e+03
...,...,...,...,...,...,...,...,...,...,...
194,2023 Q4,737.0,793.0,2.875993e+08,7.315492e+07,3.607542e+08,63.0,-1.38,15.0,-7.514234e+07
195,2024 Q1,842.0,643.0,3.411742e+08,8.617240e+07,4.273466e+08,73.0,-1.82,32.0,-8.265295e+07
196,2024 Q2,847.0,772.0,3.474049e+08,8.782984e+07,4.352347e+08,75.0,-1.70,54.0,-8.508651e+07
197,2024 Q3,812.0,690.0,4.071802e+08,1.018733e+08,5.090535e+08,75.0,-1.55,43.0,-9.318552e+07
